# Web Scraping Practice Pipeline

### Step 1: Import Required Libraries

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Step 2: Access Website

In [26]:
page_num = 3
make = 'acura'
url = f'https://www.truecar.com/used-cars-for-sale/listings/'

response = requests.get(url, timeout=5)

response.status_code

200

### Step 3: Access Used Car Listing Objects

In [27]:
soup = BeautifulSoup(response.content, 'html.parser')

results = soup.find_all('div', {'class' : 'linkable card card-shadow vehicle-card'})

len(results)

33

### Step 4: Get Individual Parameters and Output to DataFrame

In [28]:
vin = []
header = []
trim = []
price = []
mileage = []
colors = []
condition = []
location = []

for result in results:
    
    # vin
    try:
        vin.append(result.find('div', {'class':'vehicle-card-vin-carousel mt-1 text-xs'}).get_text())
    except:
        vin.append('n/a')
    # header
    try:
        header.append(result.find('div', {'class':'vehicle-card-header'}).get_text())
    except:
        header.append('n/a')
    
    # trim
    try:
        trim.append(result.find('div', {'data-test':'vehicleCardTrim'}).get_text())
    except:
        trim.append('n/a')
    # price
    try:
        price.append(result.find('div', {'class':'vehicle-card-bottom-pricing-secondary pl-3 lg:pl-2 vehicle-card-bottom-max-50'}).get_text())
    except:
        price.append('n/a')
    
    # mileage
    try:
        mileage.append(result.find('div', {'data-test':'vehicleMileage'}).get_text())
    except:
        mileage.append('n/a')
    
    # colors
    try:
        colors.append(result.find('div', {'data-test':'vehicleCardColors'}).get_text())
    except:
        colors.append('n/a')
    
    # condition
    try:
        condition.append(result.find('div', {'data-test':'vehicleCardCondition'}).get_text())
    except:
        condition.append('n/a')
    
    # location
    try:
        location.append(result.find('div', {'data-test': 'vehicleCardLocation'}).get_text())
    except:
        location.append('n/a')
        

car_listings = pd.DataFrame({'vin': vin, 'header': header, 'trim': trim, 'price': price, 'mileage': mileage, 'colors': colors, 'condition': condition, 'location': location})

### Step 5: Preview Raw Tabular Data

In [29]:
car_listings.head(5)

,vin,header,trim,price,mileage,colors,condition,location
0,VIN1FTBR1C83LKA62166,Sponsored2020 Ford Transit Cargo Van,"T-250 148"" Medium Roof 9070 GVWR RWD","$41,577","24,465 miles","White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX"
1,VIN1GNSKDKD3NR208863,Sponsored2022 Chevrolet Suburban,Z71 4WD,"$65,998","27,058 miles","Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX"
2,VIN1G1FZ6S02N4108127,Sponsored2022 Chevrolet Bolt EUV,Premier,"$33,998","2,370 miles","Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA"
3,VIN1FTEW1EB4MFA62542,2021 Ford F-150,XLT SuperCrew 5.5' Box 4WD,"$38,499","15,745 miles","White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID"
4,VIN1FTFW1E89MFA71147,2021 Ford F-150,XLT SuperCrew 5.5' Box 4WD,"$35,990","45,732 miles","Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT"


### Step 6: Extract Model Year from Listing Header

In [30]:
car_listings['year'] = np.where(car_listings['header'].str.startswith('Spon'), car_listings['header'].str[9:13], car_listings['header'].str[:5])
car_listings.head(5)

,vin,header,trim,price,mileage,colors,condition,location,year
0,VIN1FTBR1C83LKA62166,Sponsored2020 Ford Transit Cargo Van,"T-250 148"" Medium Roof 9070 GVWR RWD","$41,577","24,465 miles","White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX",2020
1,VIN1GNSKDKD3NR208863,Sponsored2022 Chevrolet Suburban,Z71 4WD,"$65,998","27,058 miles","Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX",2022
2,VIN1G1FZ6S02N4108127,Sponsored2022 Chevrolet Bolt EUV,Premier,"$33,998","2,370 miles","Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA",2022
3,VIN1FTEW1EB4MFA62542,2021 Ford F-150,XLT SuperCrew 5.5' Box 4WD,"$38,499","15,745 miles","White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID",2021
4,VIN1FTFW1E89MFA71147,2021 Ford F-150,XLT SuperCrew 5.5' Box 4WD,"$35,990","45,732 miles","Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT",2021


### Step 7: Extract Make and Model from Listing Header

In [31]:
try:
    new_cols = car_listings['header'].str.split(" ", n=5, expand=True)
    car_listings['make'] = new_cols[1]
    car_listings['model'] = new_cols[2]
    car_listings['model2'] = new_cols[3]
    car_listings['model3'] = new_cols[4]
except:
    new_cols = car_listings['header'].str.split(" ", n=4, expand=True)
    car_listings['make'] = new_cols[1]
    car_listings['model'] = new_cols[2]
    car_listings['model2'] = new_cols[3]
    
car_listings.head(5)

,vin,header,trim,price,mileage,colors,condition,location,year,make,model,model2,model3
0,VIN1FTBR1C83LKA62166,Sponsored2020 Ford Transit Cargo Van,"T-250 148"" Medium Roof 9070 GVWR RWD","$41,577","24,465 miles","White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX",2020,Ford,Transit,Cargo,Van
1,VIN1GNSKDKD3NR208863,Sponsored2022 Chevrolet Suburban,Z71 4WD,"$65,998","27,058 miles","Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX",2022,Chevrolet,Suburban,None,None
2,VIN1G1FZ6S02N4108127,Sponsored2022 Chevrolet Bolt EUV,Premier,"$33,998","2,370 miles","Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA",2022,Chevrolet,Bolt,EUV,None
3,VIN1FTEW1EB4MFA62542,2021 Ford F-150,XLT SuperCrew 5.5' Box 4WD,"$38,499","15,745 miles","White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID",2021,Ford,F-150,None,None
4,VIN1FTFW1E89MFA71147,2021 Ford F-150,XLT SuperCrew 5.5' Box 4WD,"$35,990","45,732 miles","Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT",2021,Ford,F-150,None,None


### Step 8: Drop "header" Column and Reorder Columns

In [32]:
car_listings = car_listings.drop(columns=['header'])
try:
    car_listings = car_listings[['vin', 'year', 'make', 'model', 'model2', 'model3', 'trim', 'price', 'mileage', 'colors', 'condition', 'location']]
except:
    car_listings = car_listings[['vin', 'year', 'make', 'model', 'model2', 'trim', 'price', 'mileage', 'colors', 'condition', 'location']]
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,colors,condition,location
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD","$41,577","24,465 miles","White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX"
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,"$65,998","27,058 miles","Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX"
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,"$33,998","2,370 miles","Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA"
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,"$38,499","15,745 miles","White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID"
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,"$35,990","45,732 miles","Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT"


### Step 9: Remove "$" and "," from "price" Column and Convert to Int

In [33]:
car_listings['price'] = car_listings['price'].str.replace('$', '')
car_listings['price'] = car_listings['price'].str.replace(',', '')
car_listings['price'] = car_listings['price'].astype(int)
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,colors,condition,location
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,"24,465 miles","White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX"
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,"27,058 miles","Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX"
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,"2,370 miles","Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA"
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,"15,745 miles","White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID"
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,"45,732 miles","Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT"


### Step 9: Remove "miles" and "," from "mileage" Column and Convert to Int

In [34]:
car_listings['mileage'] = car_listings['mileage'].str.replace('miles', '')
car_listings['mileage'] = car_listings['mileage'].str.replace(',', '')
car_listings['mileage'] = car_listings['mileage'].astype(int)
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,colors,condition,location
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX"
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX"
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA"
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID"
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT"


### Step 10: Split "colors" Column into "exterior_color" and "interior_color" Columns

In [35]:
new_cols = car_listings['colors'].str.split(",", n=2, expand=True)
car_listings['exterior_color'] = new_cols[0]
car_listings['interior_color'] = new_cols[1]
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,colors,condition,location,exterior_color,interior_color
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX",White exterior,Gray interior
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX",Gray exterior,Black interior
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA",Black exterior,Gray interior
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID",White exterior,Black interior
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT",Blue exterior,Black interior


### Step 11: Remove "exterior" and "interior" From "exterior_color" and "interior_color" Columns, Respectively

In [36]:
car_listings['exterior_color'] = car_listings['exterior_color'].str.replace('exterior', '')
car_listings['interior_color'] = car_listings['interior_color'].str.replace('interior', '')
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,colors,condition,location,exterior_color,interior_color
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX",White,Gray
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX",Gray,Black
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA",Black,Gray
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID",White,Black
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT",Blue,Black


### Step 12: Replace "Online" Retailer Locations with "Online" values in "location" Column

In [37]:
car_listings['location'] = car_listings['location'].str.replace('Online RetailerDelivery Available', 'Online')
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,colors,condition,location,exterior_color,interior_color
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"White exterior, Gray interior","1 accident, 1 Owner, Personal use","Lewisville, TX",White,Gray
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Gray exterior, Black interior","No accidents reported, 1 Owner, Personal use","Tomball, TX",Gray,Black
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Black exterior, Gray interior","No accidents reported, 1 Owner, Personal use","Waukee, IA",Black,Gray
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"White exterior, Black interior","No accidents reported, 1 Owner, Personal use","Blackfoot, ID",White,Black
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"Blue exterior, Black interior","1 accident, 1 Owner, Personal use","North Lake City, UT",Blue,Black


### Step 13: Drop "colors" Column and Split "condition" Column into 3 New Columns

In [38]:
car_listings = car_listings.drop(columns=['colors'])

new_cols = car_listings['condition'].str.split(", ", n=3, expand=True)
car_listings['num_accidents'] = new_cols[0]
car_listings['num_owners'] = new_cols[1]
car_listings['use_type'] = new_cols[2]

car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,condition,location,exterior_color,interior_color,num_accidents,num_owners,use_type
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"1 accident, 1 Owner, Personal use","Lewisville, TX",White,Gray,1 accident,1 Owner,Personal use
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"No accidents reported, 1 Owner, Personal use","Tomball, TX",Gray,Black,No accidents reported,1 Owner,Personal use
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"No accidents reported, 1 Owner, Personal use","Waukee, IA",Black,Gray,No accidents reported,1 Owner,Personal use
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"No accidents reported, 1 Owner, Personal use","Blackfoot, ID",White,Black,No accidents reported,1 Owner,Personal use
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"1 accident, 1 Owner, Personal use","North Lake City, UT",Blue,Black,1 accident,1 Owner,Personal use


### Step 14: Drop "condition" Column and Clean "num_accidents", "num_owners", and "use_type" Columns

In [39]:
car_listings = car_listings.drop(columns=['condition'])

car_listings['num_accidents'] = np.where(car_listings['num_accidents'].str.contains('No'), '0', car_listings['num_accidents'].str[0:1])
car_listings['num_accidents'] = car_listings['num_accidents'].str.strip()
car_listings['num_owners'] = car_listings['num_owners'].str[0:1]
car_listings['num_owners'] = car_listings['num_owners'].str.strip()
car_listings['use_type'] = car_listings['use_type'].str.replace('use', '')

car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,location,exterior_color,interior_color,num_accidents,num_owners,use_type
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"Lewisville, TX",White,Gray,1,1,Personal
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Tomball, TX",Gray,Black,0,1,Personal
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Waukee, IA",Black,Gray,0,1,Personal
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"Blackfoot, ID",White,Black,0,1,Personal
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"North Lake City, UT",Blue,Black,1,1,Personal


### Step 15: Convert "num_accidents" and "num_owners" to Int and "use_type" to Categorical

In [40]:
car_listings['num_accidents'] = car_listings['num_accidents'].astype(int)
car_listings['num_owners'] = car_listings['num_owners'].astype(int)
car_listings['use_type'] = car_listings['use_type'].astype('category')

car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,location,exterior_color,interior_color,num_accidents,num_owners,use_type
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"Lewisville, TX",White,Gray,1,1,Personal
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Tomball, TX",Gray,Black,0,1,Personal
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Waukee, IA",Black,Gray,0,1,Personal
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"Blackfoot, ID",White,Black,0,1,Personal
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"North Lake City, UT",Blue,Black,1,1,Personal


### Step 16: Split "location" into 2 Columns

In [41]:
new_cols = car_listings['location'].str.split(", ", n=2, expand=True)
car_listings['city'] = new_cols[0]
car_listings['state'] = new_cols[1]

car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,location,exterior_color,interior_color,num_accidents,num_owners,use_type,city,state
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,"Lewisville, TX",White,Gray,1,1,Personal,Lewisville,TX
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,"Tomball, TX",Gray,Black,0,1,Personal,Tomball,TX
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,"Waukee, IA",Black,Gray,0,1,Personal,Waukee,IA
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,"Blackfoot, ID",White,Black,0,1,Personal,Blackfoot,ID
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,"North Lake City, UT",Blue,Black,1,1,Personal,North Lake City,UT


### Step 17: Drop "location" column and Reorder Columns

In [42]:
car_listings = car_listings.drop(columns=['location'])

try:
    car_listings = car_listings[['vin', 'year', 'make', 'model', 'model2', 'model3', 'trim', 'price', 'mileage', 'city', 'state', 'exterior_color', 'interior_color', 'num_accidents', 'num_owners', 'use_type']]
except:
    car_listings = car_listings[['vin', 'year', 'make', 'model', 'model2', 'trim', 'price', 'mileage', 'city', 'state', 'exterior_color', 'interior_color', 'num_accidents', 'num_owners', 'use_type']]
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,city,state,exterior_color,interior_color,num_accidents,num_owners,use_type
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,Lewisville,TX,White,Gray,1,1,Personal
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,Tomball,TX,Gray,Black,0,1,Personal
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,Waukee,IA,Black,Gray,0,1,Personal
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,Blackfoot,ID,White,Black,0,1,Personal
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,North Lake City,UT,Blue,Black,1,1,Personal


### Step 18: Extract Drivetrain Data into "drivetrain" Column

In [43]:
car_listings['drivetrain'] = np.where(car_listings['trim'].str.contains('FWD'), 'FWD', np.where(car_listings['trim'].str.contains('RWD') | car_listings['trim'].str.contains('2WD'), 'RWD', np.where(car_listings['trim'].str.contains('4WD'), '4WD', np.where(car_listings['trim'].str.contains('AWD'), 'AWD', 'Unknown'))))
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,city,state,exterior_color,interior_color,num_accidents,num_owners,use_type,drivetrain
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,Lewisville,TX,White,Gray,1,1,Personal,RWD
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,None,None,Z71 4WD,65998,27058,Tomball,TX,Gray,Black,0,1,Personal,4WD
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt,EUV,None,Premier,33998,2370,Waukee,IA,Black,Gray,0,1,Personal,Unknown
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,38499,15745,Blackfoot,ID,White,Black,0,1,Personal,4WD
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,None,None,XLT SuperCrew 5.5' Box 4WD,35990,45732,North Lake City,UT,Blue,Black,1,1,Personal,4WD


### Step 19: Concatenate "model", "model2", and "model3" Columns

In [44]:
try:
    car_listings['model2'] = car_listings['model2'].fillna('')
    car_listings['model3'] = car_listings['model3'].fillna('')
    car_listings['model'] = np.where(car_listings['model3'] != None, car_listings['model'] + " " + car_listings['model2'] + " " + car_listings['model3'], np.where(car_listings['model2'] != None, car_listings['model'] + " " + car_listings['model2'], car_listings['model']))
except:
    car_listings['model2'] = car_listings['model2'].fillna('')
    car_listings['model'] = np.where(car_listings['model2'] != None, car_listings['model'] + " " + car_listings['model2'], car_listings['model'])
car_listings.head(5)

,vin,year,make,model,model2,model3,trim,price,mileage,city,state,exterior_color,interior_color,num_accidents,num_owners,use_type,drivetrain
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit Cargo Van,Cargo,Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,Lewisville,TX,White,Gray,1,1,Personal,RWD
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,,,Z71 4WD,65998,27058,Tomball,TX,Gray,Black,0,1,Personal,4WD
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt EUV,EUV,,Premier,33998,2370,Waukee,IA,Black,Gray,0,1,Personal,Unknown
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,,,XLT SuperCrew 5.5' Box 4WD,38499,15745,Blackfoot,ID,White,Black,0,1,Personal,4WD
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,,,XLT SuperCrew 5.5' Box 4WD,35990,45732,North Lake City,UT,Blue,Black,1,1,Personal,4WD


### Step 20: Remove "model2" and "model3" Columns and Extract Transmission Data into "transmission" Column

In [45]:
car_listings = car_listings.drop(columns=['model2', 'model3'])

car_listings['transmission'] = np.where(car_listings['trim'].str.contains('CVT'), 'CVT', np.where(car_listings['trim'].str.contains('Automatic'), 'Automatic', 'Unknown'))

car_listings.head(5)

,vin,year,make,model,trim,price,mileage,city,state,exterior_color,interior_color,num_accidents,num_owners,use_type,drivetrain,transmission
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit Cargo Van,"T-250 148"" Medium Roof 9070 GVWR RWD",41577,24465,Lewisville,TX,White,Gray,1,1,Personal,RWD,Unknown
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,Z71 4WD,65998,27058,Tomball,TX,Gray,Black,0,1,Personal,4WD,Unknown
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt EUV,Premier,33998,2370,Waukee,IA,Black,Gray,0,1,Personal,Unknown,Unknown
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,38499,15745,Blackfoot,ID,White,Black,0,1,Personal,4WD,Unknown
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,35990,45732,North Lake City,UT,Blue,Black,1,1,Personal,4WD,Unknown


### Step 21: Remove Drivetrain and Transmission Data from "trim" Column and Reorder Columns

In [46]:
car_listings['trim'] = car_listings['trim'].str.replace('4WD', '')
car_listings['trim'] = car_listings['trim'].str.replace('AWD', '')
car_listings['trim'] = car_listings['trim'].str.replace('FWD', '')
car_listings['trim'] = car_listings['trim'].str.replace('RWD', '')
car_listings['trim'] = car_listings['trim'].str.replace('2WD', '')
car_listings['trim'] = car_listings['trim'].str.replace('CVT', '')
car_listings['trim'] = car_listings['trim'].str.replace('Automatic', '')

car_listings = car_listings[['vin', 'year', 'make', 'model', 'trim', 'price', 'mileage', 'drivetrain', 'transmission', 'city', 'state', 'exterior_color', 'interior_color', 'num_accidents', 'num_owners', 'use_type']]
car_listings.head(10)

,vin,year,make,model,trim,price,mileage,drivetrain,transmission,city,state,exterior_color,interior_color,num_accidents,num_owners,use_type
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit Cargo Van,"T-250 148"" Medium Roof 9070 GVWR",41577,24465,RWD,Unknown,Lewisville,TX,White,Gray,1,1,Personal
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,Z71,65998,27058,4WD,Unknown,Tomball,TX,Gray,Black,0,1,Personal
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt EUV,Premier,33998,2370,Unknown,Unknown,Waukee,IA,Black,Gray,0,1,Personal
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,XLT SuperCrew 5.5' Box,38499,15745,4WD,Unknown,Blackfoot,ID,White,Black,0,1,Personal
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,XLT SuperCrew 5.5' Box,35990,45732,4WD,Unknown,North Lake City,UT,Blue,Black,1,1,Personal
5,VIN1FTEW1EG4JFD01384,2018,Ford,F-150,XLT SuperCrew 5.5' Box,17990,201264,4WD,Unknown,Wilmington,NC,Silver,Gray,0,1,Personal
6,VIN1C6SRFJT0KN560009,2019,Ram,1500,"Laramie Crew Cab 5'7"" Box",23998,163677,4WD,Unknown,Panama City,FL,Gray,Black,0,1,Personal
7,VIN1FTEW1E4XKFA85239,2019,Ford,F-150,XLT SuperCrew 5.5' Box,32500,34572,4WD,Unknown,Post Falls,ID,Red,Gray,0,1,Personal
8,VIN1FTEW1CP1KKC31807,2019,Ford,F-150,XL SuperCrew 5.5' Box,19499,163256,RWD,Unknown,Lewisville,TX,Black,Black,0,1,Personal
9,VIN1FT7W2BT4HEB30006,2017,Ford,Super Duty F-250,Lariat Crew Cab 8' Bed,29995,200922,4WD,Unknown,Lumberton,TX,White,Unknown,0,1,Personal


### Step 22: Preview Column Data Types

In [47]:
car_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   vin             33 non-null     object  
 1   year            33 non-null     object  
 2   make            33 non-null     object  
 3   model           33 non-null     object  
 4   trim            33 non-null     object  
 5   price           33 non-null     int32   
 6   mileage         33 non-null     int32   
 7   drivetrain      33 non-null     object  
 8   transmission    33 non-null     object  
 9   city            33 non-null     object  
 10  state           33 non-null     object  
 11  exterior_color  33 non-null     object  
 12  interior_color  33 non-null     object  
 13  num_accidents   33 non-null     int32   
 14  num_owners      33 non-null     int32   
 15  use_type        33 non-null     category
dtypes: category(1), int32(4), object(11)
memory usage: 3.6+ KB


### Step 23: Set Official Data Types

In [48]:
car_listings['year'] = car_listings['year'].str.strip()
car_listings['year'] = car_listings['year'].astype(int)

car_listings['drivetrain'] = car_listings['drivetrain'].astype('category')

car_listings['transmission'] = car_listings['transmission'].astype('category')

car_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   vin             33 non-null     object  
 1   year            33 non-null     int32   
 2   make            33 non-null     object  
 3   model           33 non-null     object  
 4   trim            33 non-null     object  
 5   price           33 non-null     int32   
 6   mileage         33 non-null     int32   
 7   drivetrain      33 non-null     category
 8   transmission    33 non-null     category
 9   city            33 non-null     object  
 10  state           33 non-null     object  
 11  exterior_color  33 non-null     object  
 12  interior_color  33 non-null     object  
 13  num_accidents   33 non-null     int32   
 14  num_owners      33 non-null     int32   
 15  use_type        33 non-null     category
dtypes: category(3), int32(5), object(8)
memory usage: 3.4+ KB


### Step 24: Output Final Batch

In [49]:
car_listings

,vin,year,make,model,trim,price,mileage,drivetrain,transmission,city,state,exterior_color,interior_color,num_accidents,num_owners,use_type
0,VIN1FTBR1C83LKA62166,2020,Ford,Transit Cargo Van,"T-250 148"" Medium Roof 9070 GVWR",41577,24465,RWD,Unknown,Lewisville,TX,White,Gray,1,1,Personal
1,VIN1GNSKDKD3NR208863,2022,Chevrolet,Suburban,Z71,65998,27058,4WD,Unknown,Tomball,TX,Gray,Black,0,1,Personal
2,VIN1G1FZ6S02N4108127,2022,Chevrolet,Bolt EUV,Premier,33998,2370,Unknown,Unknown,Waukee,IA,Black,Gray,0,1,Personal
3,VIN1FTEW1EB4MFA62542,2021,Ford,F-150,XLT SuperCrew 5.5' Box,38499,15745,4WD,Unknown,Blackfoot,ID,White,Black,0,1,Personal
4,VIN1FTFW1E89MFA71147,2021,Ford,F-150,XLT SuperCrew 5.5' Box,35990,45732,4WD,Unknown,North Lake City,UT,Blue,Black,1,1,Personal
5,VIN1FTEW1EG4JFD01384,2018,Ford,F-150,XLT SuperCrew 5.5' Box,17990,201264,4WD,Unknown,Wilmington,NC,Silver,Gray,0,1,Personal
6,VIN1C6SRFJT0KN560009,2019,Ram,1500,"Laramie Crew Cab 5'7"" Box",23998,163677,4WD,Unknown,Panama City,FL,Gray,Black,0,1,Personal
7,VIN1FTEW1E4XKFA85239,2019,Ford,F-150,XLT SuperCrew 5.5' Box,32500,34572,4WD,Unknown,Post Falls,ID,Red,Gray,0,1,Personal
8,VIN1FTEW1CP1KKC31807,2019,Ford,F-150,XL SuperCrew 5.5' Box,19499,163256,RWD,Unknown,Lewisville,TX,Black,Black,0,1,Personal
9,VIN1FT7W2BT4HEB30006,2017,Ford,Super Duty F-250,Lariat Crew Cab 8' Bed,29995,200922,4WD,Unknown,Lumberton,TX,White,Unknown,0,1,Personal


In [50]:
output = pd.read_csv('car_listings.csv')
outer = car_listings.merge(output, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
pd.DataFrame(anti_join)

ValueError: You are trying to merge on int32 and object columns. If you wish to proceed you should use pd.concat

## Output to Parquet File

In [ ]:
#car_listings.to_csv('car_listings.csv', index=False, mode='a', header=False)

In [ ]:
# all_listings = pd.read_csv('car_listings.csv')
# len(all_listings)

15965